In [1]:
"""
stream.py
Contains classes to create different streamers for data from the HPWREN SDSC Davis weather station
"""

import os
import sys
#import urllib.request
import requests
import urllib3 
import numpy as np
import threading
import datetime
import time
import json
from json import dumps
import gspread
from oauth2client.service_account import ServiceAccountCredentials



class SDSCWeatherStreamer(object):
    """
    Description
    """
    def __init__(self, url_base, t_period=60):
        """
        Initialize EnvStreamer object with a url_base of the raw data from a Davis sensor with proper date formatting
        in the url to interface with datetime module
        """
        self._timer = None
        self.url_base = url_base
        self.url_prev = datetime.date.today().strftime(self.url_base) # generate the date string
        self.t_period = t_period
        self.t_next_read = time.time() + self.t_period
        self.t_streamed_up_to = time.time()
        self.b_running = False

        self.start_stream()

    def _execute(self):
        """
        Function triggered by the timer every self.t_period seconds
        """
        url_curr = datetime.date.today().strftime(self.url_base)
        print('{0:0.1f}: Accessing data from {1}'.format(time.time(), url_curr))
       
        # If day has rolled over, collect all information from previous day before continuing
        if (not(url_curr == self.url_prev)):
            # I changed the import, so I change the code accordingly
            # with urllib.request.urlopen(url_curr) as fid: # shouldn't be url_prev? this is the same as the line below
            #r = http.request('GET',url_curr)
            with requests.get(url_curr) as fid:
            #with http.request('GET',url_curr) as fid:
                data_temp = fid.text
                #data_temp = fid.read()
                if (len(data_temp) > 0):
                    
                    self.process_data(data_temp)
        # I change the code accordingly
        #with urllib.request.urlopen(url_curr) as fid:
        #r = http.request('GET',url_curr)
        with requests.get(url_curr) as fid:
        #with http.request('GET',url_curr) as fid:
            data_temp = fid.text
            #data_temp = fid.read()
            if (len(data_temp) > 0):
               
                self.process_data(data_temp)


        self.url_prev = url_curr

        
   
    def process_data(self, data_in):
        """
        Processes all data from time defined in self.t_streamed_up_to to current time, uploading all information
        to a Google Sheets doc
        """
        self.t_next_read = self.t_next_read + self.t_period
        self.b_running = False
        self.start_stream()

        for line in data_in.split(b'\n'):
            #print('for loop in process_data')
            try:
                list_vars = line.split(b'\t')
                if (len(list_vars) == 4):
                    #print('if statement in try')
                    data_ip = list_vars[0]
                    data_name = list_vars[1]
                    data_timestamp = int(list_vars[2])
                    data_csv = list_vars[3].split(b',')
                else:
                    continue

                #if (data_timestamp <= self.t_streamed_up_to):
                    
                    #continue

                print('  New data: {0}'.format(data_csv))

                # do numerical operation on all the elements
                Bar = int(data_csv[8]+data_csv[7],16)/29.83
                Bar = str(Bar)

                TempOut = int(data_csv[13]+data_csv[12],16)/10
                TempOut = float(TempOut)
                TempOut = str(((100.0/(212-32) * (TempOut - 32))*100)/100)

                TempIn = int(data_csv[10]+data_csv[9],16)/10
                TempIn = float(TempIn)
                TempIn = str(((100.0/(212-32) * (TempIn - 32))*100)/100)

                HumOut = str(int(data_csv[33],16))

                HumIn = str(int(data_csv[11],16))

                Wdir = str(int(data_csv[17]+data_csv[16],16))

                Wind = str(int(data_csv[14],16)/2.23694)

                Wind10 = str(int(data_csv[15],16)/2.23694)

                Solar = str(int(data_csv[45]+data_csv[44],16))

                UV = str(int(data_csv[43],16))

                RainRate = str(int(data_csv[42]+data_csv[41],16))

                DTime = datetime.datetime.fromtimestamp(data_timestamp).isoformat(' ')
                
                
                
                values = [DTime,Bar,TempIn,HumIn,TempOut,Wind,Wind10,Wdir,HumOut,RainRate,UV,Solar]
               
                # Put in code to upload to Google Sheets
                scope = ["https://www.googleapis.com/auth/drive"]

                creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
                client = gspread.authorize(creds)

                sheet = client.open("SDSC Weather Data").sheet2 # open sheet
                
               
                sheet.append_row(values)
                print('put data into Google Sheet')
                # Update most recently read data
                self.t_streamed_up_to = data_timestamp

            except Exception as e:
                print(e)
                print('ERROR: {0}'.format(line))
                print(sys.exc_info()[0])

    def start_stream(self):
        """
        Start the stream
        """
        if (not(self.b_running)):
            t_wait = self.t_next_read - time.time()
            if (t_wait < 0.1):
                t_wait = 1
            elif (t_wait > self.t_period):
                t_wait = self.t_period
            self._timer = threading.Timer(t_wait, self._execute)
            self._timer.start()
            self.b_running = True

    def stop_stream(self):
        """
        Stop the stream
        """
        self._timer.cancel()
        self.b_running = False




In [2]:
if __name__ == '__main__':
    t_period = 5
    url = 'http://hpwren.ucsd.edu/TM/Sensors/Data/20190223/198.202.124.3-HPWREN:SDSC:Davis:1:0'
    sdsc_weather_streamer = SDSCWeatherStreamer(url_base=url, t_period=t_period)

    try:
        while (True):
            time.sleep(1)

    finally:
        sdsc_weather_streamer.stop_stream()

1551133058.0: Accessing data from http://hpwren.ucsd.edu/TM/Sensors/Data/20190223/198.202.124.3-HPWREN:SDSC:Davis:1:0
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'10', u'05', u'67', u'75', u'a5', u'02', u'1a', u'c6', u'01', u'00', u'01', u'83', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'46', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'00', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'34', u'03', u'08', u'01', u'ef', u'05', u'ed', u'00', u'0a', u'0d', u'10', u'71']
'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550908800	4c,4f,4f,14,00,10,05,67,75,a5,02,1a,c6,01,00,01,83,00,ff,ff,ff,ff,ff,ff,ff,

  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'10', u'05', u'67', u'75', u'a5', u'02', u'1a', u'c6', u'01', u'00', u'01', u'83', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'46', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'00', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'34', u'03', u'08', u'01', u'ef', u'05', u'ed', u'00', u'0a', u'0d', u'10', u'71']
'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550908871	4c,4f,4f,14,00,11,05,67,75,a5,02,19,c6,01,00,01,83,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,46,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,00,00,ca,00,cf,01,ff,ff,f

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550908831	4c,4f,4f,14,00,10,05,67,75,a5,02,1a,c6,01,00,01,83,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,46,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,00,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,08,01,ef,05,ed,00,0a,0d,10,71
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'10', u'05', u'67', u'75', u'a5', u'02', u'1a', u'c6', u'01', u'00', u'01', u'83', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'46', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'00', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550908871	4c,4f,4f,14,00,11,05,67,75,a5,02,19,c6,01,00,01,83,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,46,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,00,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,08,01,ef,05,ed,00,0a,0d,4c,ae
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'11', u'05', u'67', u'75', u'a5', u'02', u'19', u'c6', u'01', u'02', u'01', u'83', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'46', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'00', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550908902	4c,4f,4f,14,00,11,05,67,75,a5,02,19,c7,01,01,01,83,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,46,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,00,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,08,01,ef,05,ed,00,0a,0d,af,5c
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'12', u'05', u'67', u'75', u'a5', u'02', u'1a', u'c7', u'01', u'01', u'01', u'83', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'46', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'00', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

...

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550916793	4c,4f,4f,14,00,95,05,7b,75,9b,02,19,c4,01,00,01,9e,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,47,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,02,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,06,2c,ef,05,ed,00,0a,0d,d3,2c
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'95', u'05', u'7b', u'75', u'9b', u'02', u'19', u'c4', u'01', u'01', u'01', u'9e', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'47', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'02', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550916823	4c,4f,4f,14,00,95,05,7b,75,9b,02,19,c4,01,02,01,9e,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,47,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,02,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,06,2c,ef,05,ed,00,0a,0d,1e,f7
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'96', u'05', u'79', u'75', u'9b', u'02', u'19', u'c4', u'01', u'02', u'01', u'9e', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'47', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'02', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550916854	4c,4f,4f,14,00,96,05,79,75,9b,02,19,c4,01,01,01,9e,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,47,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,02,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,06,2c,ef,05,ed,00,0a,0d,b7,c3
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'96', u'05', u'79', u'75', u'9b', u'02', u'19', u'c4', u'01', u'02', u'01', u'9e', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'47', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'02', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550916996	4c,4f,4f,14,00,98,05,79,75,9b,02,18,c3,01,01,01,9e,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,47,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,02,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,06,2c,ef,05,ed,00,0a,0d,c0,1c
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'98', u'05', u'79', u'75', u'9b', u'02', u'18', u'c1', u'01', u'01', u'01', u'9e', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'47', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'02', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u

'Spreadsheet' object has no attribute 'sheet2'
ERROR: 198.202.124.3	HPWREN:SDSC:Davis:1:0	1550917026	4c,4f,4f,14,00,99,05,79,75,9b,02,19,c1,01,02,01,9e,00,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,ff,47,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,ff,ff,00,00,89,01,08,05,02,00,ca,00,cf,01,ff,ff,ff,ff,ff,ff,ff,ff,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,34,03,06,2c,ef,05,ed,00,0a,0d,08,80
<type 'exceptions.AttributeError'>
  New data: [u'4c', u'4f', u'4f', u'14', u'00', u'99', u'05', u'79', u'75', u'9b', u'02', u'19', u'c1', u'01', u'01', u'01', u'9e', u'00', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'47', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'ff', u'ff', u'00', u'00', u'89', u'01', u'08', u'05', u'02', u'00', u'ca', u'00', u'cf', u'01', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'ff', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u'00', u